## Challenge 1

In [1]:
import datetime
import csv
from collections import defaultdict
from pprint import pprint
import dateutil.parser
import datetime


In [2]:
#we are interested in all of May 2016 and the first two weeks of June
#However it is extremely important that we select the "correct" dates
#according to the MTA URL
start_date = datetime.date(2016,5,7)
end_date = datetime.date(2016,6,11)
link_prefix = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_"
urls = []

#it's important to use datetime objects and not just enter the dates as strings
#so that we can perform timedelta
def date_format(date):
    return date.strftime("%y%m%d")

this_date = start_date
while this_date <= end_date:
    urls.append(link_prefix + date_format(this_date) + ".txt")
    this_date += datetime.timedelta(days=7)


In [3]:
#run bash cmd to wget and download files to current dir
for f in urls:
    !wget {f}b

--2017-01-19 21:01:13--  http://web.mta.info/developers/data/nyct/turnstile/turnstile_160507.txtb
Resolving web.mta.info (web.mta.info)... 23.215.104.57, 23.215.104.24
Connecting to web.mta.info (web.mta.info)|23.215.104.57|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2017-01-19 21:01:14 ERROR 404: Not Found.

--2017-01-19 21:01:14--  http://web.mta.info/developers/data/nyct/turnstile/turnstile_160514.txtb
Resolving web.mta.info (web.mta.info)... 23.215.104.57, 23.215.104.24
Connecting to web.mta.info (web.mta.info)|23.215.104.57|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2017-01-19 21:01:14 ERROR 404: Not Found.

--2017-01-19 21:01:14--  http://web.mta.info/developers/data/nyct/turnstile/turnstile_160521.txtb
Resolving web.mta.info (web.mta.info)... 23.215.104.24, 23.215.104.57
Connecting to web.mta.info (web.mta.info)|23.215.104.24|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2017-01-19 21:01:14 ERROR 

In [4]:
#store the files for reference
files = !ls | grep turnstile.*txt
working_file = files[0]

In [5]:
#make a dictionary with the first four columns as keys
reader_file = csv.reader(open(working_file, 'r'))
next(reader_file)

#create a function that accepts a csvreader obj and returns a dictionary
#key is the first 4 columns, value is the rest
def build_dictionary(reader):
    dictionary = defaultdict(list)
    for row in reader:
        key = (row[0], row[1], row[2], row[3])
        value = [row[4], row[5], row[6], row[7], row[8], row[9], row[10]]
        dictionary[key] = value
    return dictionary

data = build_dictionary(reader_file)
keys = list(data.keys()) #python 3 returns 'views' for d.keys(), so we have to store as a list to iterate
pprint ({k: data[k] for k in keys[:5]})

{('H009', 'R235', '00-00-01', 'BEDFORD AV'): ['L',
                                              'BMT',
                                              '05/06/2016',
                                              '20:00:00',
                                              'REGULAR',
                                              '0000128450',
                                              '0000141149                                             '],
 ('N083', 'R138', '01-06-01', 'W 4 ST-WASH SQ'): ['ABCDEFM',
                                                  'IND',
                                                  '05/06/2016',
                                                  '21:00:00',
                                                  'REGULAR',
                                                  '0015350226',
                                                  '0004557868                                   '],
 ('PTH07', 'R550', '00-01-00', 'CITY / BUS'): ['1',
                                  

## Challenge 2

In [8]:
#make a dictionary with the first four columns as keys
reader_file = csv.reader(open(working_file, 'r'))
next(reader_file)

def build_timeseries(reader):
    dictionary = defaultdict(list)
    for row in reader:
        key = (row[0], row[1], row[2], row[3])
        string = row[6]+' '+row[7] #store the date as a string
        date = dateutil.parser.parse(string)
        num_entries = int(row[9]) #store number of entries
        value = [date, num_entries]
        dictionary[key] = value
    return dictionary

time_series = build_timeseries(reader_file)
time_series_keys = list(time_series.keys())
pprint ({k: time_series[k] for k in time_series_keys[:5]})



{('N063A', 'R011', '00-00-06', '42 ST-PORT AUTH'): [datetime.datetime(2016, 5, 6, 20, 0),
                                                    1411169433],
 ('N076', 'R111', '00-06-01', '23 ST'): [datetime.datetime(2016, 5, 6, 20, 0),
                                         815284],
 ('N137', 'R354', '00-06-01', '104 ST'): [datetime.datetime(2016, 5, 6, 20, 0),
                                          1681399520],
 ('N412', 'R299', '00-00-01', 'BROADWAY'): [datetime.datetime(2016, 5, 6, 21, 0),
                                            4943681],
 ('R644', 'R135', '01-00-01', 'NEWKIRK AV'): [datetime.datetime(2016, 5, 6, 21, 0),
                                              215687]}
